In [ ]:
# mubawab_scraper_spark_incremental.py

import os
import re
import csv
import json
import time
import random
from typing import List, Dict, Any
from urllib.parse import urljoin
from datetime import datetime

import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# =========================================================
# 1. ENV & SPARK CONFIG
# =========================================================

load_dotenv()

storage_account = "strealestatehamza"
container = "realestate"

adls_key = os.getenv("ADLS_ACCOUNT_KEY")
if not adls_key:
    raise RuntimeError("ADLS_ACCOUNT_KEY missing from .env")

spark = (
    SparkSession
    .builder
    .appName("RealEstate_Mubawab_Docker")
    .getOrCreate()
)

# Set ADLS account keys (DFS + BLOB)
spark.conf.set(
    f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
    adls_key,
)
spark.conf.set(
    f"fs.azure.account.key.{storage_account}.blob.core.windows.net",
    adls_key,
)

# =========================================================
# 2. CONSTANTS & BASIC UTILS
# =========================================================

BASE_URL = "https://www.mubawab.ma"

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0 Safari/537.36"
    )
}


def fetch_html(url: str) -> str:
    """Télécharge le HTML d'une page Mubawab."""
    resp = requests.get(url, headers=HEADERS, timeout=20)
    resp.raise_for_status()
    return resp.text


def extract_id_from_url(url: str) -> str:
    """
    Extrait l'ID numérique Mubawab depuis une URL du type:
      https://www.mubawab.ma/fr/a/8256920/...
    """
    m = re.search(r"/a/(\d+)", url.split("?")[0])
    return m.group(1) if m else ""


# =========================================================
# 3. BREADCRUMBS, CAT, LOCATION, DATE
# =========================================================

def get_breadcrumbs(soup: BeautifulSoup) -> Dict[str, Any]:
    """
    Retourne breadcrumbs_list et breadcrumbs (string).
    Extrait depuis le div.adBread selon le HTML Mubawab.
    """
    crumbs: List[str] = []

    # 1) Chercher le div.adBread spécifique à Mubawab
    ad_bread = soup.find("div", class_=re.compile(r"adBread"))
    if ad_bread:
        for a in ad_bread.find_all("a", class_="darkblue", href=True):
            text = a.get_text(strip=True)
            if text:
                crumbs.append(text)

    # 2) Fallback: ul.breadcrumb li a
    if not crumbs:
        ul = soup.find("ul", class_=re.compile(r"breadcrumb"))
        if ul:
            for li in ul.find_all("li"):
                a = li.find("a")
                text = (a or li).get_text(strip=True)
                if text:
                    crumbs.append(text)

    # 3) Fallback: nav[aria-label="Fil d'ariane"]
    if not crumbs:
        nav = soup.find("nav", attrs={"aria-label": re.compile("Fil", re.I)})
        if nav:
            for li in nav.find_all("li"):
                text = li.get_text(strip=True)
                if text:
                    crumbs.append(text)

    return {
        "breadcrumbs_list": crumbs,
        "breadcrumbs": " > ".join(crumbs) if crumbs else "",
    }


def get_category_label(soup: BeautifulSoup) -> str:
    """
    Récupère un libellé de catégorie :
        ex: 'Villa, à vendre' ou 'Appartement, à louer'
    """
    cat_label = ""
    type_bien = ""

    # Récupération du bloc Caractéristiques générales
    for feature in soup.select("div.caractBlockProp div.adMainFeature"):
        label_el = feature.select_one("p.adMainFeatureContentLabel")
        value_el = feature.select_one("p.adMainFeatureContentValue")
        if not label_el or not value_el:
            continue
        label = label_el.get_text(strip=True)
        value = value_el.get_text(strip=True)

        if "Type de bien" in label:
            type_bien = value
            break

    # Essai de deviner louer/vendre à partir du texte de page
    full_text = soup.get_text(" ", strip=True).lower()
    suffix = ""
    if "à louer" in full_text or "a louer" in full_text:
        suffix = ", à louer"
    elif "à vendre" in full_text or "a vendre" in full_text:
        suffix = ", à vendre"

    if type_bien:
        cat_label = type_bien + suffix
    return cat_label


def get_location_and_date(soup: BeautifulSoup) -> Dict[str, str]:
    """
    Récupère:
      - location: ex 'Hay Targa à Marrakech'
      - published_date: date/heure de scraping (maintenant)
      - scraping_time: timestamp exact du scraping
    """
    location = ""

    # Location: ex <h3 class="greyTit">Guéliz à Marrakech</h3>
    grey = soup.find("h3", class_=re.compile(r"greyTit"))
    if grey:
        location = grey.get_text(" ", strip=True)

    # Date de publication = temps de scraping
    scraping_time = datetime.utcnow()
    published_date = scraping_time.strftime("%Y-%m-%d %H:%M:%S")

    return {
        "location": location,
        "published_date": published_date,
        "scraping_time": scraping_time.isoformat(),
    }


# =========================================================
# 4. TITLE, PRICE, DESCRIPTION, IMAGES
# =========================================================

def get_title_and_price(soup: BeautifulSoup) -> Dict[str, str]:
    """
    Récupère le titre (h1.searchTitle) et le prix (h3.orangeTit).
    """
    title = ""
    price_text = ""

    # Titre
    h1 = soup.find("h1", class_=re.compile(r"searchTitle"))
    if h1:
        title = h1.get_text(strip=True)

    # Prix : h3.orangeTit
    price_block = soup.find("h3", class_=re.compile(r"orangeTit"))
    if price_block:
        price_text = price_block.get_text(strip=True)

    return {
        "title": title,
        "price_text": price_text,
    }


def get_description(soup: BeautifulSoup) -> str:
    """
    Récupère la description dans le bloc:
      <div class="blockProp"><h1 class="searchTitle">..</h1><p>...</p></div>
    """
    desc = ""

    # Bloc contenant la description (celui qui a un <h1 class="searchTitle">)
    for block in soup.find_all("div", class_=re.compile(r"blockProp")):
        h1 = block.find("h1", class_=re.compile(r"searchTitle"))
        if h1:
            p = block.find("p")
            if p:
                desc = p.get_text(" ", strip=True)
            break

    if not desc:
        # fallback: premier paragraphe long
        p = soup.find("p")
        if p:
            desc = p.get_text(" ", strip=True)

    desc = re.sub(r"\s+", " ", desc).strip()
    return desc


def get_images(soup: BeautifulSoup) -> List[str]:
    """
    Récupère toutes les URLs d'images de la galerie principale Mubawab.
    On cible les URLs 'mubawab-media.com/ad/...'..
    """
    urls: List[str] = []

    # Overlay principale
    for img in soup.select("div.overlayPhoto img[src]"):
        src = img.get("src")
        if src and "mubawab-media.com/ad/" in src and src not in urls:
            urls.append(src)

    # Galerie
    for img in soup.select("#picturesGallery img[src]"):
        src = img.get("src")
        if src and "mubawab-media.com/ad/" in src and src not in urls:
            urls.append(src)

    # Slider
    for img in soup.select("#picturesSlider img[src]"):
        src = img.get("src")
        if src and "mubawab-media.com/ad/" in src and src not in urls:
            urls.append(src)

    # Fallback générique
    for img in soup.find_all("img", src=True):
        src = img["src"]
        if "mubawab-media.com/ad/" in src and src not in urls:
            urls.append(src)

    return urls


# =========================================================
# 5. ATTRIBUTES, EQUIPMENTS, SELLER INFO
# =========================================================

def get_attributes_and_equipments(soup: BeautifulSoup) -> Dict[str, Any]:
    """
    Récupère :
      - attributes: dict { "Type de bien": "Villa", "Etat": "Bon état", ... }
      - equipments: liste ["Terrasse", "Piscine", "Climatisation", ...]
    """
    attributes: Dict[str, str] = {}
    equipments: List[str] = []

    # 1) Caractéristiques générales
    for feature in soup.select("div.caractBlockProp div.adMainFeature"):
        label_el = feature.select_one("p.adMainFeatureContentLabel")
        value_el = feature.select_one("p.adMainFeatureContentValue")
        if not label_el or not value_el:
            continue
        label = label_el.get_text(" ", strip=True)
        value = value_el.get_text(" ", strip=True)
        if label and value:
            attributes[label] = value

    # 2) Détails surface / SDB / etc. dans le header
    for span in soup.select("div.adDetails div.adDetailFeature span"):
        txt = span.get_text(" ", strip=True)
        if not txt:
            continue

        if "m²" in txt or "m2" in txt:
            key = "Surface"
        elif "Salle de bain" in txt or "Salles de bains" in txt:
            key = "Salle de bain"
        elif "Pièce" in txt:
            key = "Pièces"
        else:
            key = txt

        attributes[key] = txt

    # 3) Coordonnées + locationId/locationType
    lat_el = soup.select_one("input#latField")
    lng_el = soup.select_one("input#lngField")
    loc_id_el = soup.select_one("input#locationId")
    loc_type_el = soup.select_one("input#locationType")

    if lat_el and lat_el.get("value"):
        attributes["Latitude"] = lat_el["value"].strip()
    if lng_el and lng_el.get("value"):
        attributes["Longitude"] = lng_el["value"].strip()
    if loc_id_el and loc_id_el.get("value"):
        attributes["Location ID"] = loc_id_el["value"].strip()
    if loc_type_el and loc_type_el.get("value"):
        attributes["Location Type"] = loc_type_el["value"].strip()

    # 4) Équipements
    for feat in soup.select("div.caractBlockProp div.adFeatures div.adFeature span"):
        txt = feat.get_text(" ", strip=True)
        if txt:
            equipments.append(txt)

    return {
        "attributes": attributes,
        "equipments": equipments,
    }


def get_seller_info(soup: BeautifulSoup) -> Dict[str, Any]:
    """
    Récupère seller_name, seller_url, seller_is_store (Agence vs Particulier).
    """
    seller_name = ""
    seller_url = ""
    seller_is_store = False  # True si agence / pro

    business_info = soup.select_one("div.businessInfo")
    if business_info:
        name_el = business_info.select_one("span.businessName")
        if name_el:
            full_txt = name_el.get_text(" ", strip=True)
            txt_lower = full_txt.lower()

            if "particulier" in txt_lower:
                seller_is_store = False
                seller_name = (
                    full_txt.replace("Particulier", "")
                    .replace("particulier", "")
                    .strip(" -|,")
                )
            elif "agence" in txt_lower:
                seller_is_store = True
                seller_name = (
                    full_txt.replace("Agence", "")
                    .replace("agence", "")
                    .strip(" -|,")
                )
            else:
                seller_name = full_txt

        # lien vers la page agence / pro s'il existe
        a = business_info.find("a", href=True)
        if a:
            seller_url = urljoin(BASE_URL, a["href"])

    return {
        "seller_name": seller_name,
        "seller_url": seller_url,
        "seller_is_store": seller_is_store,
    }


# =========================================================
# 6. PARSER PRINCIPAL
# =========================================================

def parse_mubawab_ad(url: str) -> Dict[str, Any]:
    """
    Scrape une annonce Mubawab (page /fr/a/...) et retourne un dict.
    """
    html = fetch_html(url)
    soup = BeautifulSoup(html, "html.parser")

    ad_id = extract_id_from_url(url)

    crumbs = get_breadcrumbs(soup)
    cat_label = get_category_label(soup)
    loc_date = get_location_and_date(soup)
    title_price = get_title_and_price(soup)
    description = get_description(soup)
    images = get_images(soup)
    attrs_equip = get_attributes_and_equipments(soup)
    seller = get_seller_info(soup)

    ad_data: Dict[str, Any] = {
        "id": ad_id,
        "url": url,
        "title": title_price["title"],
        "price_text": title_price["price_text"],
        "location": loc_date["location"],
        "published_date": loc_date["published_date"],
        "scraping_time": loc_date["scraping_time"],
        "breadcrumbs_list": crumbs["breadcrumbs_list"],
        "breadcrumbs": crumbs["breadcrumbs"],
        "category_label": cat_label,
        "description": description,
        "images": images,
        "attributes": attrs_equip["attributes"],
        "equipments": attrs_equip["equipments"],
        "seller_name": seller["seller_name"],
        "seller_url": seller["seller_url"],
        "seller_is_store": seller["seller_is_store"],
    }

    return ad_data


# =========================================================
# 7. LISTING -> URLS (PAGINATION)
# =========================================================

def get_ad_urls_from_listing(listing_url: str, page: int = 1) -> List[str]:
    """
    Récupère toutes les URLs d'annonces d'une page listing Mubawab.
    Pagination réelle Mubawab: :p:2, :p:3, ...
    Exemple:
      page 1: https://.../villa-sale
      page 2: https://.../villa-sale:p:2
    """
    # On enlève un éventuel suffixe :p:XX pour être robuste
    base_url = re.sub(r":p:\d+$", "", listing_url)

    if page == 1:
        page_url = base_url
    else:
        page_url = f"{base_url}:p:{page}"

    html = fetch_html(page_url)
    soup = BeautifulSoup(html, "html.parser")

    ad_urls = set()

    for a in soup.find_all("a", href=True):
        href = a["href"].strip()

        # Lien relatif → absolu
        if href.startswith("/"):
            href = urljoin(BASE_URL, href)

        # Garder seulement les liens Mubawab
        if "mubawab.ma" not in href:
            continue

        # Garder seulement les liens d'annonces /fr/a/
        if "/fr/a/" in href:
            clean = href.split("?")[0]
            ad_urls.add(clean)

    ad_urls = sorted(ad_urls)
    print(f"🌐 Trouvé {len(ad_urls)} annonces sur {page_url}")
    return ad_urls



# =========================================================
# 8. NORMALISATION POUR SPARK
# =========================================================

def normalize_ads_for_spark(ads: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Convertit les champs dict/list en strings pour Spark (OK aussi pour Parquet).
    """
    normalized: List[Dict[str, Any]] = []
    for ad in ads:
        d = ad.copy()
        d["attributes"] = json.dumps(d.get("attributes", {}), ensure_ascii=False)
        d["equipments"] = "; ".join(d.get("equipments", []) or [])
        d["breadcrumbs_list"] = json.dumps(
            d.get("breadcrumbs_list", []) or [],
            ensure_ascii=False
        )
        d["images"] = ", ".join(d.get("images", []) or [])
        normalized.append(d)
    return normalized


# =========================================================
# 9. PROCESSUS INCRÉMENTAL PAGE PAR PAGE
# =========================================================

def process_listing_incremental(
    listing_url: str,
    subfolder: str,         # "ventes" ou "locations"
    offre_label: str,       # "vente" ou "location"
    base_path: str,
    max_pages: int = 100,
):
    """
    Scrape page par page un listing Mubawab et écrit chaque page en Parquet
    dans ADLS, avant de passer à la page suivante.
    """
    seen_urls = set()
    now_iso = datetime.utcnow().isoformat()

    # Même date_path pour toutes les pages de ce run, pour ce listing
    date_path = datetime.utcnow().strftime("%Y/%m/%d/%H%M%S")

    for page in range(1, max_pages + 1):
        print(f"\n========== {subfolder.upper()} – PAGE {page}/{max_pages} ==========")

        try:
            ad_urls = get_ad_urls_from_listing(listing_url, page=page)
        except Exception as e:
            print(f"❌ Erreur lors du chargement de la page {page}: {e}")
            break

        if not ad_urls:
            print("Aucune annonce trouvée sur cette page, arrêt du scraping pour ce listing.")
            break

        # éviter les doublons entre pages
        new_urls = [u for u in ad_urls if u not in seen_urls]
        if not new_urls:
            print("Toutes les annonces de cette page sont déjà vues, arrêt du scraping pour ce listing.")
            break

        seen_urls.update(new_urls)
        total = len(new_urls)

        page_ads: List[Dict[str, Any]] = []

        for i, ad_url in enumerate(new_urls, 1):
            print(f"[{subfolder}][Page {page}] [{i}/{total}] Scraping {ad_url}")
            try:
                ad_data = parse_mubawab_ad(ad_url)
                page_ads.append(ad_data)
            except Exception as e:
                print(f"  ❌ Erreur sur {ad_url}: {e}")

            time.sleep(random.uniform(1.0, 2.5))  # respect du site

        if not page_ads:
            print(f"⚠️ Aucun ad valide sur cette page {page}, on continue à la suivante.")
            continue

        # Normalisation
        normalized_ads = normalize_ads_for_spark(page_ads)

        # DataFrame Spark pour cette page
        df = spark.createDataFrame(normalized_ads)

        df = (
            df
            .withColumn("source_site", F.lit("mubawab"))
            .withColumn("offre", F.lit(offre_label))
            .withColumn("ingest_ts", F.lit(now_iso))
        )

        print(f"📌 Schéma {subfolder} - page {page}")
        df.printSchema()

        # Path page par page : .../mubawab/ventes/{date_path}/page=001
        output_path = (
            f"{base_path}/mubawab/{subfolder}/{date_path}/page={page:03d}"
        )

        (
            df
            .coalesce(1)
            .write
            .mode("overwrite")
            .parquet(output_path)
        )
        print(f"✅ Mubawab {subfolder.capitalize()} PAGE {page} (PARQUET) -> {output_path}")


# =========================================================
# 10. MAIN WORKFLOW: INCRÉMENTAL
# =========================================================

def main():
    # ---- Listing URLs ----
    ventes_url = (
        "https://www.mubawab.ma/fr/cc/"
        "immobilier-a-vendre-all:o:n:sc:"
        "apartment-sale,commercial-sale,farm-sale,house-sale,land-sale,"
        "office-sale,other-sale,riad-sale,villa-sale"
    )

    locations_url = (
        "https://www.mubawab.ma/fr/cc/"
        "immobilier-a-louer-all:o:n:sc:"
        "apartment-rent,commercial-rent,farm-rent,house-rent,land-rent,"
        "office-rent,other-rent,riad-rent,room-rent,villa-rent"
    )

    base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/raw"

    # ---- Ventes: 100 pages max, page par page ----
    process_listing_incremental(
        listing_url=ventes_url,
        subfolder="ventes",
        offre_label="vente",
        base_path=base_path,
        max_pages=100,
    )

    # ---- Locations: 100 pages max, page par page ----
    process_listing_incremental(
        listing_url=locations_url,
        subfolder="locations",
        offre_label="location",
        base_path=base_path,
        max_pages=100,
    )


if __name__ == "__main__":
    main()


========== VENTES – PAGE 1/100 ==========
🌐 Trouvé 33 annonces sur https://www.mubawab.ma/fr/cc/immobilier-a-vendre-all:o:n:sc:apartment-sale,commercial-sale,farm-sale,house-sale,land-sale,office-sale,other-sale,riad-sale,villa-sale
[ventes][Page 1] [1/33] Scraping https://www.mubawab.ma/fr/a/8258297/villa-d%E2%80%99architecte-%C3%A0-vendre-moderne-avec-vue-mer-ain-diab
[ventes][Page 1] [2/33] Scraping https://www.mubawab.ma/fr/a/8258303/appartement-%C3%A0-l-achat-%C3%A0-moujahidine-3-pi%C3%A8ces-terrasse-et-ascenseur
[ventes][Page 1] [3/33] Scraping https://www.mubawab.ma/fr/a/8258307/appartement-haut-standing-84m-casablanca-errahma
[ventes][Page 1] [4/33] Scraping https://www.mubawab.ma/fr/a/8258310/superbe-appartement-%C3%A0-vendre-%C3%A0-ibrahim-al-khalil-superficie-86-m%C2%B2-prestation-de-conciergerie-air-conditionn%C3%A9
[ventes][Page 1] [5/33] Scraping https://www.mubawab.ma/fr/a/8258314/appartement-%C3%A0-vendre-%C3%A0-hay-al-bassatine-2-chambres-agr%C3%A9ables-cuisine-%C3%A9